# Machine Learning

In [1]:
import warnings

import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense
from keras.metrics import TopKCategoricalAccuracy
from keras.models import Sequential, load_model
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import ndcg_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.
C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
train = pd.read_csv('./data/train_df.csv', index_col=0)

In [3]:
le = LabelEncoder()
train['target'] = le.fit_transform(train['country_destination'])
lb = LabelBinarizer()
lb.fit(train['target'])

LabelBinarizer()

In [4]:
mapping = dict(zip(range(len(le.classes_)), le.classes_))
mapping

{0: 'AU',
 1: 'CA',
 2: 'DE',
 3: 'ES',
 4: 'FR',
 5: 'GB',
 6: 'IT',
 7: 'NDF',
 8: 'NL',
 9: 'PT',
 10: 'US',
 11: 'other'}

In [5]:
target = train['target']
feature = train.drop(['target', 'country_destination'], axis=1)

In [6]:
print(target.shape, feature.shape)

(213451,) (213451, 125)


### Baseline Model

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    feature, target, train_size=.25, random_state=42)

In [8]:
def model_train(name, reg):
    fit = reg.fit(X_train, y_train)
    score = fit.predict_proba(X_test)
    ndcg = ndcg_score(lb.transform(y_test), score, k=5)
    print('{} has ndcg score of {:.3f}'.format(name, ndcg))

In [9]:
def base_model():
    model = Sequential()
    model.add(Dense(240, input_dim=125, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(12, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy', 'top_k_categorical_accuracy'])
    return model

In [10]:
estimator = KerasClassifier(
    build_fn=base_model, epochs=15, batch_size=128, verbose=0)
kfold = KFold(n_splits=4, shuffle=True)
results = cross_val_score(estimator, X_train, lb.transform(
    y_train), cv=kfold, scoring='roc_auc_ovr')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [11]:
print("Baseline roc_auc_ovr score: %.2f%%" % (results.mean()*100))

Baseline roc_auc_ovr score: 63.80%


In [12]:
estimator.fit(feature, lb.transform(target), verbose=False)

In [13]:
estimator.model.save(filepath='./data/deep_learn.tf')

In [14]:
estimator.fit(X_train, y_train)
score = estimator.predict_proba(X_test)
ndcg = ndcg_score(lb.transform(y_test), score, k=5)
print('model has ndcg score of {:.3f}'.format(ndcg))

model has ndcg score of 0.824
